In [19]:
from pathlib import Path

import psutil

In [20]:
pids = {}
for p in psutil.process_iter():
    if p.name() == "python3":
        pids[p.pid] = p
len(pids)

In [33]:
for pid in pids:
    p = pids[pid]
    try:
        print(p)
        p = Path(f"/proc/{p.pid}/cwd")
        print(list(p.iterdir())[:3])
        print()
    except Exception as e:
        print(e)

In [22]:
pid = 216468

In [23]:
process = pids[pid]

In [24]:
process.cwd()

In [25]:
process.cmdline()

In [26]:
process.connections()

In [27]:
heap = None
stack = None
for mm in process.memory_maps(False):
    print(mm)
    if heap is None:
        if mm.path == "[heap]":
            heap = mm
            break
    if stack is None:
        if mm.path == "[stack]":
            stack = mm
            break

In [28]:
stack

In [29]:
heap.addr

In [30]:
memory_file = Path(f"/proc/{pid}/mem")

In [31]:
memory = memory_file.read_bytes()

In [34]:
#! /usr/bin/env python
#
# Modification from:
# https://unix.stackexchange.com/questions/6267/how-to-re-load-all-running-applications-from-swap-space-into-ram/6271
#
# For non-root check 'cat /proc/sys/kernel/yama/ptrace_scope' = 0
# Reference: https://www.kernel.org/doc/Documentation/security/Yama.txt
#
import ctypes
import re
import sys

c_ptrace = ctypes.CDLL("libc.so.6").ptrace
c_pid_t = ctypes.c_int32  # This assumes pid_t is int32_t
c_ptrace.argtypes = [ctypes.c_int, c_pid_t, ctypes.c_void_p, ctypes.c_void_p]
PTRACE_ATTACH = 16
PTRACE_DETACH = 17


def ptrace(attach, pid):
    op = ctypes.c_int(PTRACE_ATTACH if attach else PTRACE_DETACH)
    c_pid = c_pid_t(pid)
    null = ctypes.c_void_p()
    err = c_ptrace(op, c_pid, null, null)
    if err != 0:
        raise RuntimeError("ptrace", err)


ptrace(True, pid)
mappath = "/proc/%s/maps" % (pid)
mempath = "/proc/%s/mem" % (pid)
outpath = "/tmp/dump.dat"
with open(mappath, "r") as maps_file:
    memmap = maps_file.readlines()
mem_file = open(mempath, "rb", 0)
out_file = open(outpath, "wb")
i = 0
for line in memmap:
    m = re.match(r"([0-9A-Fa-f]+)-([0-9A-Fa-f]+) ([-r])", line)
    if m.group(3) == "r":
        start = int(m.group(1), 16)
        end = int(m.group(2), 16)
        try:
            pos = mem_file.seek(start)
            chunk = mem_file.read(end - start)
            print(("Read Line: %s" % (line)), file=sys.stderr)
        except:
            print(("Could not read: %016x" % (pos)), file=sys.stderr)
            print(("Line: %s" % (line)), file=sys.stderr)
            pass
        result = out_file.write(chunk)
print(("Wrote file: %s" % (outpath)), file=sys.stderr)
out_file.close()
mem_file.close()
ptrace(False, pid)

In [ ]:
import time

from ptrace.binding import *  # For ptrace funcs
from ptrace.linux_proc import *  # For the searchProcessByName func


def checkVal(value):
    # Check value bounds and such here
    if value is good:
        return True


def main():
    pid = searchProcessByName("nameofprocess")  # or pid = 56437
    addr = 0x32323232  # Note: poke/peek_text requires the address to be aligned
    while True:  # This may require some modifications to your new value as well
        newVal = input("What do you want the new value to be?")
        if checkVal(newVal):
            ptrace_attach(pid)  # Attach
            time.sleep(0.001)  # For some reason, I needed this for it to work
            ptrace_peektext(pid, addr)  # Read word at addr
            ptrace_poketext(pid, addr, newVal)  # Write newVal at addr
            ptrace_detach(pid)  # Let the process resume